In [2]:
# import the general stuff
import os
from os import getcwd
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn import metrics

#import stuff for gradCAM
import matplotlib.cm as cm
from IPython.display import Image

#import the tf stuff
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Check TensorFlow Version
print('TF version: {}' .format(tf.__version__))

#Check for GPU utilization
if tf.test.gpu_device_name():
    print(' GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU")

TF version: 2.3.0
 GPU: /device:GPU:0


In [3]:
#set Image size (RGB so imshape is 3)
pix_dim = 416
imsize = (pix_dim,pix_dim) 
imshape = (pix_dim,pix_dim,3)

In [4]:
#load model
filepath = './saved_VGG16_retrain_slower'
model = tf.keras.models.load_model(filepath, compile = True)

In [5]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 416, 416, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 416, 416, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 416, 416, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 208, 208, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 208, 208, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 208, 208, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 104, 104, 128)    

In [6]:
#test the model

#define test directory
test_dir_wash = '../data/testing_michael/wash/'
test_dir_nowash = '../data/testing_michael/nowash'

#load images for wash
w_test = []
w_images = []
for img in os.listdir(test_dir_wash):
    #pull the name of the image
    w_images.append(img)
    #pull the image
    img = os.path.join(test_dir_wash, img)
    img = tf.keras.preprocessing.image.load_img(img,target_size = imsize)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img/255
    img = np.expand_dims(img,axis=0)
    w_test.append(img)

#load images for nowash
nw_test = []
nw_images = []
for img in os.listdir(test_dir_nowash):
    #pull the name of the image
    nw_images.append(img)
    #pull the image
    img = os.path.join(test_dir_nowash, img)
    img = tf.keras.preprocessing.image.load_img(img,target_size = imsize)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img/255
    img = np.expand_dims(img,axis=0)
    nw_test.append(img)

#make predictions 
w_test = np.vstack(w_test)
w_preds = model.predict(w_test)

nw_test = np.vstack(nw_test)
nw_preds = model.predict(nw_test)

#stack up the predicitons and truth in the same format

#vstack pred array
all_pred = np.vstack((w_preds,nw_preds))
all_pred = all_pred > 0.5

#vstack true array
all_obs = np.vstack((np.full((70,1), True), np.full((70,1), False)))

#report relevant binary classification stats for test data
report = metrics.classification_report(all_obs, all_pred)
print(report)

              precision    recall  f1-score   support

       False       0.93      0.91      0.92        70
        True       0.92      0.93      0.92        70

    accuracy                           0.92       140
   macro avg       0.92      0.92      0.92       140
weighted avg       0.92      0.92      0.92       140



In [7]:
#and the confusion matrix for test data
CM = metrics.confusion_matrix(all_obs, all_pred)
CM

array([[64,  6],
       [ 5, 65]])

In [8]:
#check the 4 categories of the Conf. Matrix
tn, fp, fn, tp = CM.ravel()
(tn, fp, fn, tp)

(64, 6, 5, 65)

In [9]:
#make labels into an array (wash then nowash)
nw_images = np.vstack((np.asarray(nw_images)))
w_images = np.vstack((np.asarray(w_images)))
all_images = np.vstack((w_images, nw_images))

#make single df for image names, true class, and predicted class 
#data = np.hstack((all_labels, all_true, all_pred))
results_df = pd.DataFrame(np.hstack((all_images, all_obs, all_pred)), columns=["Image", "Observed", "Predicted"])
results_df.head()

,Image,Observed,Predicted
0,P28462771.jpg,True,True
1,P28462380.jpg,True,True
2,P28463002.jpg,True,True
3,P28463062.jpg,True,True
4,P28463052.jpg,True,True


In [10]:
#save the df as a csv
results_df.to_csv('../data/VGG16_Michael_results.csv', index=False)